In [29]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

checkpoint_path = "/workspace/whisper-train/checkpoint-9000"
model_id = "openai/whisper-small"
output_dir = "./checkpoints/whisper-small/"

In [30]:
device

'cuda'

### 데이터셋 불러오기 & 데이터 로더 만들기

In [31]:
from datasets import load_from_disk
import os

# 저장된 경로
train_path = "/workspace/train/"
test_path = "/workspace/test/"

# 데이터셋 로드
train_dataset = load_from_disk(train_path)
test_dataset = load_from_disk(test_path)

print(f"✅ Train 데이터셋 로드 완료: {len(train_dataset)}개")
print(f"✅ Test 데이터셋 로드 완료: {len(test_dataset)}개")

# 확인
print(f"\n📊 Train Dataset:")
print(train_dataset)
print(f"\n컬럼: {train_dataset.column_names}")
print(f"Features: {train_dataset.features}")

Loading dataset from disk:   0%|          | 0/65 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

✅ Train 데이터셋 로드 완료: 33694개
✅ Test 데이터셋 로드 완료: 8414개

📊 Train Dataset:
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 33694
})

컬럼: ['input_features', 'labels']
Features: {'input_features': List(List(Value('float32'))), 'labels': List(Value('int64'))}


In [32]:
train_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 33694
})

In [33]:
test_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 8414
})

In [40]:
dataset_split = train_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

In [41]:
train_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 24259
})

In [42]:
eval_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 6065
})

In [52]:
from transformers import (
    AutoProcessor,
    AutoModelForSpeechSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype="float32",
    low_cpu_mem_usage=True,
    use_safetensors=True,
).to(device)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [53]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import torch
import numpy as np

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    
    def __call__(self, features: List[Dict[str, Union[List[int], np.ndarray, torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # input_features 처리
        input_features = []
        for f in features:
            feat = f["input_features"]
            if isinstance(feat, torch.Tensor):
                input_features.append(feat.detach().clone().float())
            elif isinstance(feat, np.ndarray):
                input_features.append(torch.from_numpy(feat).float())
            else:
                input_features.append(torch.FloatTensor(feat))
        
        batch = {"input_features": torch.stack(input_features)}
        
        # labels 처리
        labels = []
        for f in features:
            label = f["labels"]
            if isinstance(label, torch.Tensor):
                labels.append(label.detach().clone().long())
            elif isinstance(label, np.ndarray):
                labels.append(torch.from_numpy(label).long())
            elif isinstance(label, list):
                labels.append(torch.LongTensor(label))
            else:
                labels.append(torch.LongTensor(list(label)))
        
        # 패딩
        from torch.nn.utils.rnn import pad_sequence
        labels = pad_sequence(labels, batch_first=True, padding_value=-100)
        
        batch["labels"] = labels
        return batch

# 4. DataCollator 생성
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [54]:
from torch.utils.data import DataLoader

# 4. DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
eval_loader = DataLoader(eval_dataset, batch_size=8, shuffle=False, collate_fn=data_collator)

In [55]:
# 5. 확인
for batch in train_loader:
    print(f"\n✅ 배치 로드 성공!")
    print(f"Input: {batch['input_features'].shape}")
    print(f"Labels: {batch['labels'].shape}")
    break


✅ 배치 로드 성공!
Input: torch.Size([8, 80, 3000])
Labels: torch.Size([8, 22])


#### 모델 로드

In [56]:
model

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

### 평가 지표

In [57]:
import evaluate
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


### 파라미터 정의

In [ ]:
# 에러 방지: generation config 명시적으로 설정
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []

model.gradient_checkpointing_enable()

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-train-1029",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-6,
    warmup_steps=500,
    max_steps=20000,  # step 기반
    
    bf16=True,
    fp16=False,
    gradient_checkpointing=False,
    
    # Step 기반 평가
    eval_strategy="steps",  # steps 기반 -> 에러 방지
    eval_steps=500,         # 500 step마다 평가
    save_strategy="steps",
    save_steps=500,
    logging_steps=50,
    
    predict_with_generate=True,
    generation_max_length=225,
    
    optim="adamw_torch",
    max_grad_norm=0.3,
    weight_decay=0.01,
    
    report_to=["tensorboard"],
    dataloader_num_workers=0,
    
    # Early Stopping 설정
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # eval_loss 기준
    greater_is_better=False,
)

# Early Stopping 콜백 추가
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],  # 5번 연속 개선 없으면 중단
)

/tmp/ipykernel_819/787002183.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
print("early stop 적용 학습")
trainer.train()
trainer.save_model("./whisper-small-best")
print("✅ 완료!")

early stop 적용 학습


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,0.669300,0.642538
1000,0.282900,0.265044
1500,0.245600,0.244027
2000,0.213300,0.233343
2500,0.195200,0.224390
3000,0.194600,0.216845
3500,0.155800,0.212044
4000,0.152300,0.207813
4500,0.170400,0.203508
5000,0.144400,0.200882


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid cra